In [18]:
from dotenv import load_dotenv
import os
import requests
import pandas as pd

In [53]:
def get_county_data(release=2024):
    """
    Retrieve county-level health-risk behaviors and health outcomes data from The CDC PLACES API.
    
    Parameters
    ----------
    release : int
        The version of release to retrieve from.

    Returns
    -------
    county_df : pandas DataFrame
        A dataframe containing information of county-level PLACES data
    
    Examples
    --------
    >>> df = get_county_data(release=2023)
    >>> df.head()
    """
    url_2024 = 'https://data.cdc.gov/api/v3/views/swc5-untb/query.json'
    url_2023 = 'https://data.cdc.gov/api/v3/views/h3ej-a9ec/query.json'
    url_2022 = 'https://data.cdc.gov/api/v3/views/duw2-7jbt/query.json'
    url_2021 = 'https://data.cdc.gov/api/v3/views/pqpp-u99h/query.json'
    
    if release == 2023:
        url = url_2023
    elif release == 2022:
        url = url_2022
    elif release == 2021:
        url = url_2021
    else:
        url = url_2024
    
    load_dotenv()
    api_token = os.getenv('CDC_API_TOKEN')
    headers = {'X-App-Token': api_token}
    params = {}

    response = requests.get(url, headers=headers, params=params)
    print(f"Status Code: {response.status_code}")

    data = response.json()
    county_df = pd.DataFrame(data)
    county_df = county_df.iloc[:, 4:]
    county_df = county_df[county_df['category'].isin(['Health Outcomes', 'Health Risk Behaviors'])]
    return county_df

In [39]:
county_data_2023 = get_county_data(2023)
county_data_2023.head()

Status Code: 200


,year,stateabbr,statedesc,locationname,datasource,category,measure,data_value_unit,data_value_type,data_value,low_confidence_limit,high_confidence_limit,totalpopulation,locationid,categoryid,measureid,datavaluetypeid,short_question_text,geolocation
0,2021,TX,Texas,Potter,BRFSS,Health Outcomes,Current asthma among adults aged >=18 years,%,Crude prevalence,10.1,8.9,11.5,116547,48375,HLTHOUT,CASTHMA,CrdPrv,Current Asthma,"{'type': 'Point', 'coordinates': [-101.893804,..."
1,2021,TN,Tennessee,Trousdale,BRFSS,Health Outcomes,Arthritis among adults aged >=18 years,%,Crude prevalence,25.4,21.8,29.1,12035,47169,HLTHOUT,ARTHRITIS,CrdPrv,Arthritis,"{'type': 'Point', 'coordinates': [-86.1566909,..."
2,2021,TX,Texas,Shackelford,BRFSS,Health Outcomes,Arthritis among adults aged >=18 years,%,Crude prevalence,29.0,26.5,31.7,3212,48417,HLTHOUT,ARTHRITIS,CrdPrv,Arthritis,"{'type': 'Point', 'coordinates': [-99.3470045,..."
4,2020,UT,Utah,Summit,BRFSS,Health Outcomes,All teeth lost among adults aged >=65 years,%,Crude prevalence,6.3,3.5,9.9,43093,49043,HLTHOUT,TEETHLOST,CrdPrv,All Teeth Lost,"{'type': 'Point', 'coordinates': [-110.9684862..."
5,2021,VA,Virginia,Amelia,BRFSS,Health Outcomes,Stroke among adults aged >=18 years,%,Crude prevalence,4.2,3.7,4.8,13268,51007,HLTHOUT,STROKE,CrdPrv,Stroke,"{'type': 'Point', 'coordinates': [-77.9774619,..."


In [51]:
def get_measure_by_state(df, measure = None, state = None):
    if measure == None or state == None:
        return "Missing parameters."
    else:
        result = df[(df['measureid'] == measure) & (df['stateabbr'] == state)]
        return result

In [52]:
get_measure_by_state(county_data_2023, measure='CASTHMA', state='TX')

,year,stateabbr,statedesc,locationname,datasource,category,measure,data_value_unit,data_value_type,data_value,low_confidence_limit,high_confidence_limit,totalpopulation,locationid,categoryid,measureid,datavaluetypeid,short_question_text,geolocation
0,2021,TX,Texas,Potter,BRFSS,Health Outcomes,Current asthma among adults aged >=18 years,%,Crude prevalence,10.1,8.9,11.5,116547,48375,HLTHOUT,CASTHMA,CrdPrv,Current Asthma,"{'type': 'Point', 'coordinates': [-101.893804,..."
164750,2021,TX,Texas,Tarrant,BRFSS,Health Outcomes,Current asthma among adults aged >=18 years,%,Crude prevalence,9.7,8.6,10.9,2126477,48439,HLTHOUT,CASTHMA,CrdPrv,Current Asthma,"{'type': 'Point', 'coordinates': [-97.2912241,..."
164900,2021,TX,Texas,Montgomery,BRFSS,Health Outcomes,Current asthma among adults aged >=18 years,%,Age-adjusted prevalence,9.3,8.1,10.6,648886,48339,HLTHOUT,CASTHMA,AgeAdjPrv,Current Asthma,"{'type': 'Point', 'coordinates': [-95.50295, 3..."
165287,2021,TX,Texas,Hall,BRFSS,Health Outcomes,Current asthma among adults aged >=18 years,%,Crude prevalence,9.8,8.6,11.0,2845,48191,HLTHOUT,CASTHMA,CrdPrv,Current Asthma,"{'type': 'Point', 'coordinates': [-100.5763434..."
165388,2021,TX,Texas,Brewster,BRFSS,Health Outcomes,Current asthma among adults aged >=18 years,%,Age-adjusted prevalence,8.8,7.7,10.1,9450,48043,HLTHOUT,CASTHMA,AgeAdjPrv,Current Asthma,"{'type': 'Point', 'coordinates': [-103.2524579..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
228031,2021,TX,Texas,Delta,BRFSS,Health Outcomes,Current asthma among adults aged >=18 years,%,Age-adjusted prevalence,10.2,8.9,11.5,5392,48119,HLTHOUT,CASTHMA,AgeAdjPrv,Current Asthma,"{'type': 'Point', 'coordinates': [-95.6733504,..."
228160,2021,TX,Texas,McLennan,BRFSS,Health Outcomes,Current asthma among adults aged >=18 years,%,Crude prevalence,10.1,8.9,11.5,263115,48309,HLTHOUT,CASTHMA,CrdPrv,Current Asthma,"{'type': 'Point', 'coordinates': [-97.2015108,..."
228215,2021,TX,Texas,Ward,BRFSS,Health Outcomes,Current asthma among adults aged >=18 years,%,Crude prevalence,9.6,8.4,10.8,11194,48475,HLTHOUT,CASTHMA,CrdPrv,Current Asthma,"{'type': 'Point', 'coordinates': [-103.1051127..."
228512,2021,TX,Texas,Swisher,BRFSS,Health Outcomes,Current asthma among adults aged >=18 years,%,Crude prevalence,10.1,8.8,11.4,7008,48437,HLTHOUT,CASTHMA,CrdPrv,Current Asthma,"{'type': 'Point', 'coordinates': [-101.7438647..."
